In [129]:
import pandas as pd
import pyreadr
import plotly.express as px
import plotly.graph_objects as go
import random

https://mqscores.lsa.umich.edu/measures.php

In [65]:

df = pd.read_csv("justices.csv")

In [66]:
df

,term,justice,justiceName,post_mn,post_sd,post_med,post_025,post_975
0,2020,117,ACBarrett,1.219,0.363,1.201,0.557,1.975
1,2021,117,ACBarrett,1.347,0.349,1.326,0.717,2.085
2,1965,97,AFortas,-1.330,0.264,-1.321,-1.869,-0.827
3,1966,97,AFortas,-1.513,0.224,-1.508,-1.971,-1.096
4,1967,97,AFortas,-1.353,0.227,-1.347,-1.818,-0.930
...,...,...,...,...,...,...,...,...
768,1971,81,WODouglas,-7.798,1.026,-7.761,-10.037,-5.885
769,1972,81,WODouglas,-7.828,1.060,-7.769,-10.110,-5.847
770,1973,81,WODouglas,-7.870,1.090,-7.818,-10.197,-5.842
771,1974,81,WODouglas,-7.923,1.116,-7.874,-10.298,-5.838


In [67]:
px.line(df, x="term", y="post_med", color="justiceName", title="Number of opinions by justice over time")

In [68]:
averges = df.groupby(["term"]).median().reset_index()
averges

C:\Users\zeusg\AppData\Local\Temp/ipykernel_52584/405081297.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,term,justice,post_mn,post_sd,post_med,post_025,post_975
0,1937,74.5,-0.4825,0.3385,-0.4775,-0.9935,-0.0055
1,1938,75.5,-0.6525,0.3155,-0.6435,-1.1820,-0.1605
2,1939,78.0,-1.0540,0.2530,-1.0470,-1.5340,-0.6090
3,1940,78.0,-0.7270,0.2700,-0.7220,-1.1430,-0.3310
4,1941,80.0,-0.1910,0.2330,-0.1910,-0.6470,0.2650
...,...,...,...,...,...,...,...
80,2017,111.0,0.4020,0.3050,0.3990,-0.0510,0.8420
81,2018,112.0,0.4510,0.3220,0.4500,0.0240,0.8930
82,2019,112.0,0.3880,0.3410,0.3840,-0.0450,0.8390
83,2020,113.0,0.6840,0.3630,0.6820,0.1990,1.1900


In [244]:
#Get the justice nears to the median for each term
with_median= df.merge(averges, on="term", suffixes=("", "_median"))
with_median["diff"] = with_median["post_med"] - with_median["post_med_median"]

above_median = with_median[with_median["diff"] > 0]
above_median = above_median.groupby(["term"]).min().reset_index()
below_median = with_median[with_median["diff"] < 0]
below_median = below_median.groupby(["term"]).max().reset_index()


In [239]:
fig = go.Figure()
df_modern = df#[df["term"] > 1950]
averges = df_modern.groupby(["term"]).median().reset_index()
for justice in df_modern["justiceName"].unique():
    data = df_modern[df_modern["justiceName"] == justice]
    fig.add_trace(go.Scatter(x=data["term"], y=data["post_med"], mode='lines', name=justice))
fig.add_trace(go.Scatter(x=[1937, 2021], y=[0, 0], mode='lines',line= dict(color='red', width=4, dash='dash'), name='zero'))
fig.add_trace(go.Scatter(x=averges["term"], y=averges["post_med"], mode='lines', name='median', line= dict(color='black', width=4)))

#width and height 

fig.update_layout(

    width=1200,
    height=800,
)
fig.show()

C:\Users\zeusg\AppData\Local\Temp/ipykernel_52584/3030790058.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [240]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=above_median["term"], y=above_median["post_med"], mode='lines', name='above median', line= dict(color='blue', width=4))) 
fig.add_trace(go.Scatter(x=below_median["term"], y=below_median["post_med"], mode='lines', name='below median', line= dict(color='red', width=4)))
fig.add_trace(go.Scatter(x=averges["term"], y=averges["post_med"], mode='lines', name='median', line= dict(color='black', width=4)))
fig.add_trace(go.Scatter(x=[1937, 2021], y=[0, 0], mode='lines',line= dict(color='red', width=4, dash='dash'), name='zero'))
fig.show()

In [241]:
# Add presidential terms 
presidents = pd.read_csv("presidents.csv")
#Split years in two columns
presidents["start"] = presidents["Years In Office"].apply(lambda x: int(x.split("-")[0]))
#Set end date as start of next row
presidents["end"] = presidents["start"].shift(-1)
presidents["end"].iloc[-1] = 2021
presidents["end"] = presidents["end"].astype(int)

C:\Users\zeusg\AppData\Local\Temp/ipykernel_52584/594050292.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [242]:
presidents
presidents = presidents[presidents["start"]>1932]
presidents
presidents["start"].iloc[0]= 1937

In [305]:
#Smooth y values
smooth_averges = averges.copy()
smooth_averges["post_med"] = smooth_averges["post_med"].rolling(window=2).mean()

smooth_above_median = above_median.copy()
smooth_above_median["post_med"] = smooth_above_median["post_med"].rolling(window=2).mean()

smooth_below_median = below_median.copy()
smooth_below_median["post_med"] = smooth_below_median["post_med"].rolling(window=2).mean()


#Fill in area by president part
fig = go.Figure()
fig.add_trace(go.Scatter(y=[1937, 2021], x=[0, 0], mode='lines',line= dict(color='yellow', width=8, dash='dash'), name='Centrist'))
fig.add_trace(go.Scatter(y=smooth_above_median["term"], x=smooth_above_median["post_med"], mode='lines', name='above median', line= dict(color='red', width=4)))
fig.add_trace(go.Scatter(y=smooth_below_median["term"], x=smooth_below_median["post_med"], mode='lines', name='below median', line= dict(color='blue', width=4)))
fig.add_trace(go.Scatter(y=smooth_averges["term"], x=smooth_averges["post_med"], mode='lines', name='median', line= dict(color='black', width=4)))


for i, row in presidents.iterrows():
    if row["Party"] == "Democratic":
        color = "blue"
    else:
        color = "red"
    fig.add_shape(
        type="rect",
        y0=row["start"],
        x0=smooth_below_median["post_med"].min(),
        y1=row["end"],
        x1=smooth_above_median["post_med"].max(),
        line=dict(
            color=color,
            width=0,
        ),
        fillcolor=color,
        opacity=0.15,
    )
    
    offset = 0
    if i % 2 == 0:
        offset = .99
    
    fig.add_hline(y=row["end"], line_width=2, line_color="black", opacity=0.5, line_dash="dash")
    
    fig.add_annotation(
        yref = 'y',
        xref='paper',
        #Add a little offset to avoid overlapping
        x=0+offset,
        y=row["start"]+ (row["end"]-row["start"])/2,
        text="<b>" + row["President Name"].replace(" ", "<br>"),
        showarrow=False,
        font=dict(
            color="black",
            size=20,
            family="Arial",
            
        )

    )
    
    #Set scale based on min and max values
    fig.update_xaxes(range=[smooth_below_median["post_med"].min(), smooth_above_median["post_med"].max()])


    
    #change height and width
fig.update_layout( 
    width=600,
    height=1100,
)
#remove x axis text
fig.update_yaxes(showticklabels=False)
fig.update_yaxes(showgrid=False, zeroline=False)

#Replace y ticks 
fig.update_xaxes(tickvals=[-2, -1, 0, 1, 2], ticktext=["", "Liberal", "", "Conservative", ""], tickfont=dict(size=20, family="Arial", color="black"))
#Rotate y ticks
#fig.update_xaxes(tickangle=270)
#update tick font


#change padding
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor="white",
    plot_bgcolor="white",
)
#Remove legend
fig.update_layout(showlegend=False)
#Reverse y axis
fig.update_yaxes(autorange="reversed")
    
fig.show()